In [1]:
!pip install ensemble_boxes

In [1]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [2]:
# ensemble csv files
submission_files = ['./work_dirs/detectors_r50_1x_trash/fold0/submission_13.csv', './work_dirs/detectors_r50_1x_trash/fold1/submission_13.csv', './work_dirs/detectors_r50_1x_trash/fold2/submission_13.csv', './work_dirs/detectors_r50_1x_trash/fold3/submission_13.csv', './work_dirs/detectors_r50_1x_trash/fold4/submission_13.csv']
submission_df = [pd.read_csv(file) for file in submission_files]

In [3]:
image_ids = submission_df[0]['image_id'].tolist()

In [4]:
annotation = '../../input/data/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [5]:
prediction_strings = []
file_names = []
iou_thr = 0.4

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        if len(predict_list)==0 or len(predict_list)==1:
            continue
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        boxes, scores, labels = nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(label) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

Warning. Removed 20 boxes with zero area!
Warning. Removed 133 boxes with zero area!
Warning. Removed 186 boxes with zero area!
Warning. Removed 128 boxes with zero area!
Warning. Removed 191 boxes with zero area!
Warning. Removed 68 boxes with zero area!
Warning. Removed 308 boxes with zero area!
Warning. Removed 19 boxes with zero area!
Warning. Removed 10 boxes with zero area!
Warning. Removed 30 boxes with zero area!
Warning. Removed 10 boxes with zero area!
Warning. Removed 94 boxes with zero area!
Warning. Removed 30 boxes with zero area!
Warning. Removed 103 boxes with zero area!
Warning. Removed 31 boxes with zero area!
Warning. Removed 109 boxes with zero area!
Warning. Removed 41 boxes with zero area!
Warning. Removed 40 boxes with zero area!
Warning. Removed 20 boxes with zero area!
Warning. Removed 20 boxes with zero area!
Warning. Removed 40 boxes with zero area!
Warning. Removed 70 boxes with zero area!
Warning. Removed 138 boxes with zero area!
Warning. Removed 58 boxes 

In [6]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('submission_detectors_ensemble.csv')

submission.head()

,PredictionString,image_id
0,1 0.72210383 229.75542 159.67238 249.41478 239...,batch_01_vt/0021.jpg
1,1 0.6267114 286.20703 344.20764 307.75403 376....,batch_01_vt/0028.jpg
2,1 0.989523 74.59343 169.44882 370.31476 371.36...,batch_01_vt/0031.jpg
3,1 0.152076 0.0 223.42656 35.80619 312.60675 2 ...,batch_01_vt/0032.jpg
4,1 0.9641026 388.48404 263.69965 488.6741 344.9...,batch_01_vt/0070.jpg
